In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))
import matplotlib.pyplot as plt
import seaborn as sns

from keras import models
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import cv2
import random

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

# 1. EDA

In [ ]:
train_df = pd.read_csv('../input/fashion-mnist_train.csv')
test_df = pd.read_csv('../input/fashion-mnist_test.csv')
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head()

In [ ]:
feat_cols = train_df.columns.tolist()[1:]

In [ ]:
image_df = train_df[feat_cols].values.reshape(-1,28,28,1)
image_df.shape

In [ ]:
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(image_df[i][:,:,0], cmap='Greys')
    plt.tight_layout()
#     plt.show() # if show(), subplot doesn't work

# 2. Modeling - CNN

In [ ]:
X_train = train_df[feat_cols].values.reshape(-1,28,28,1)/255
X_test = test_df[feat_cols].values.reshape(-1,28,28,1)/255
y_train = np_utils.to_categorical(train_df['label'].values)
y_test = np_utils.to_categorical(test_df['label'].values)

In [ ]:
X_test[0].shape

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3, input_shape=(28,28,1), padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(2))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(2))
model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(2))
model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train, validation_data=(X_test,y_test),batch_size=128, epochs=10, verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.title('Loss')
plt.legend(loc='best')
plt.xlabel('epochs')
plt.ylabel('loss')

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.title('Accuracy')
plt.legend(loc='best')
plt.xlabel('epochs')
plt.ylabel('loss')

# 3.  Summary

[Numpy.nonzero() 详解 numpy module中 nonzero（）函数](https://blog.csdn.net/roler_/article/details/42395393)
```
>>> x = np.array([[1,0,0], [0,2,0], [1,1,0]])
>>> x
array([[1, 0, 0],
       [0, 2, 0],
       [1, 1, 0]])
>>> np.nonzero(x)
(array([0, 1, 2, 2]), array([0, 1, 0, 1])) # non zero row index array, no zero column index array
```

```
>>> a = np.array([[1,2,3],[4,5,6],[7,8,9]])
>>> a > 3
array([[False, False, False],
       [ True,  True,  True],
       [ True,  True,  True]])
>>> np.nonzero(a > 3)
(array([1, 1, 1, 2, 2, 2]), array([0, 1, 2, 0, 1, 2]))
```

**one-hot -> label**

In [ ]:
y_true = y_test
print(y_true.shape)

In [ ]:
np.nonzero(y_true)

In [ ]:
y_predict = model.predict_classes(X_test)
y_true = np.nonzero(y_true)[1]
print(y_predict.shape)
print(y_true.shape)

In [ ]:
y_predict

In [ ]:
y_true

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predict, target_names=['class {}'.format(i) for i in range(10)]))

# 精度(precision) = 正确预测的个数(TP)/被预测正确的个数(TP+FP)
# 召回率(recall)=正确预测的个数(TP)/预测个数(TP+FN)
# F1 = 2*精度*召回率/(精度+召回率)

**get index of correct / incoreect**

In [ ]:
correct = np.nonzero(y_true == y_predict)[0]
incorrect = np.nonzero(y_true != y_predict)[0]
incorrect[:5]

In [ ]:
for i, loc in enumerate(incorrect[:6]):
#     print('incorrect predict img location in test set', loc)
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[loc][:,:,0], cmap='Greys')
    plt.title('predict_{} vs true_{}'.format(y_predict[loc], y_true[loc]))
    plt.tight_layout()
# can not use ```for i in incorrect[:6] ```, because i is in []5,7,53,51,...
# when subplot will lead to i+1 > 3*3

In [ ]:
model.summary()

# Visualization

In [ ]:
print(len(model.layers))
model.layers

In [ ]:
print(model.layers[0].name)
model.layers[0].output

In [ ]:
model.input

In [ ]:
from keras import models
models.Model(input=model.input, output=model.layers[0].output)

**model.layers[0],  1st conv layer,  to show image of train[1]**

In [ ]:
activation_model = models.Model(input=model.input, output=model.layers[0].output)
activations = activation_model.predict(X_train[1].reshape(1,28,28,1))
activations.shape

In [ ]:
activations[0].shape

In [ ]:
plt.imshow(X_train[1][:,:,0])

In [ ]:
size = 32
plt.figure(figsize=(10,10))
for i in range(size): # i in range(32)，kernel size
    plt.subplot(size/8,8,i+1)
    plt.imshow(activations[0][:,:,i]) # plt.matshow() cannot be shown in subplots
    plt.title('kernel {}'.format(i))
    plt.tight_layout()

In [ ]:
# show conv & maxpooling in all CNN
for i in model.layers[:9]: # first 9 layers are conv neural network
    print(i)
    model_layer = Model(inputs=model.input, outputs=i.output)
    layer_activation = model_layer.predict(X_train[1].reshape(1,28,28,1))
    print(layer_activation.shape)
    kernel_size = layer_activation.shape[-1]
    plt.figure(figsize=(50,50))
    for j in range(kernel_size):
        plt.subplot(kernel_size/8, 8, j+1)
        plt.imshow(layer_activation[0][:,:,j])
        plt.title('{} - kernel_{}'.format(i.name, j))
        plt.tight_layout()
    print('---------'*10)